In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate


In [2]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True


### 代码解释

#### 1. `os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'`
- **作用**：
  - 设置环境变量 `HF_ENDPOINT`，将 Hugging Face 的数据集和模型下载源从官方地址 `https://huggingface.co` 切换到镜像地址 `https://hf-mirror.com`。
- **为什么需要设置？**
  - **加速下载**：`hf-mirror.com` 是 Hugging Face 官方提供的镜像站点（通常托管在亚洲地区），下载速度比官方源更快。
  - **避免网络限制**：某些地区（如中国大陆）访问 `huggingface.co` 可能较慢或不稳定，使用镜像站点可以绕过这些问题。
  - **备用方案**：如果官方源暂时不可用（如维护或宕机），镜像站点可以作为替代。

#### 2. `dataset = datasets.load_dataset("bentrevett/multi30k")`
- **作用**：
  - 从 Hugging Face Datasets 库加载名为 `"bentrevett/multi30k"` 的数据集，并存储在变量 `dataset` 中。
- **具体流程**：
  1. 检查本地缓存（默认路径：`~/.cache/huggingface/datasets/`），如果数据集已存在，则直接加载。
  2. 如果本地没有缓存，则从 `https://hf-mirror.com/datasets/bentrevett/multi30k` 下载数据集（因为之前设置了 `HF_ENDPOINT`）。
  3. 返回一个 `datasets.DatasetDict` 或 `datasets.Dataset` 对象，可以像 Pandas DataFrame 一样操作。

#### 注意事项
- **数据集名称**：`"bentrevett/multi30k"` 是 Hugging Face Datasets 上的一个数据集，由用户 `bentrevett` 上传，名称为 `multi30k`。
- **镜像源的可靠性**：虽然镜像站点通常可靠，但如果遇到问题，可以尝试其他镜像源（如阿里云镜像 `https://mirror.aliyun.com/huggingface/`）或直接使用官方源。

In [3]:
import os
from datasets import load_dataset

# 1. 基础修改方式
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 2. 加载数据集（自动走镜像）
dataset = load_dataset("multi30k")


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [5]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

In [6]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

In [7]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

In [8]:
string = "What a lovely day it is today!"

[token.text for token in en_nlp.tokenizer(string)]

['What', 'a', 'lovely', 'day', 'it', 'is', 'today', '!']

### `tokenize_example` 函数的作用

该函数用于对输入的示例数据进行分词（tokenization）和预处理，主要步骤如下：

1. **输入参数**：
   - `example`：包含源语言（英语 `"en"`）和目标语言（德语 `"de"`）文本的字典。
   - `en_nlp` 和 `de_nlp`：分别用于英语和德语的分词器（如 spaCy 的 `Tokenizer`）。
   - `max_length`：限制分词后的最大长度，超出部分会被截断。
   - `lower`：是否将分词结果转换为小写。
   - `sos_token` 和 `eos_token`：分别表示句子开始（Start of Sentence）和句子结束（End of Sentence）的特殊标记。

2. **处理流程**：
   - 使用 `en_nlp.tokenizer` 对英语文本分词，并截取前 `max_length` 个词。
   - 使用 `de_nlp.tokenizer` 对德语文本分词，并截取前 `max_length` 个词。
   - 如果 `lower=True`，则将所有分词结果转换为小写。
   - 在英语和德语的分词结果前后分别添加 `<sos>` 和 `<eos>` 标记。
   - 返回一个字典，包含处理后的英语和德语分词结果（`en_tokens` 和 `de_tokens`）。

3. **返回值**：
   - 返回一个字典，格式为：
     ```python
     {
         "en_tokens": [sos_token, token1, token2, ..., eos_token],
         "de_tokens": [sos_token, token1, token2, ..., eos_token]
     }
     ```

In [9]:
def tokenize_example(example, en_nlp, de_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens": en_tokens, "de_tokens": de_tokens}

### `<sos>` 和 `<eos>` 的含义

- **`<sos>`（Start of Sentence）**：
  - 表示句子的开始，通常用于序列到序列（Seq2Seq）模型的输入，帮助模型识别句子的起始位置。
  - 在解码阶段（如机器翻译），`<sos>` 可以作为解码的初始标记。

- **`<eos>`（End of Sentence）**：
  - 表示句子的结束，用于标记句子的终止位置。
  - 在训练时，`<eos>` 可以帮助模型学习句子的长度；在解码时，`<eos>` 可以作为停止条件。

---

### `map` 函数的作用

`map` 是 Hugging Face `datasets` 库中的一个方法，用于对数据集中的每个样本应用指定的函数（如 `tokenize_example`），并返回处理后的新数据集。具体作用如下：

1. **功能**：
   - 对 `train_data`、`valid_data` 和 `test_data` 中的每个样本调用 `tokenize_example` 函数。
   - 将原始文本数据转换为分词后的 token 序列（`en_tokens` 和 `de_tokens`）。

2. **参数**：
   - `tokenize_example`：要应用的函数。
   - `fn_kwargs`：传递给 `tokenize_example` 的额外参数（如分词器、最大长度等）。

3. **返回值**：
   - 返回一个新的数据集，其中每个样本已被 `tokenize_example` 处理。

4. **示例**：
   - 假设原始数据为：
     ```python
     {"en": "Hello world", "de": "Hallo Welt"}
     ```
   - 经过 `map` 处理后，可能变为：
     ```python
     {
         "en_tokens": ["<sos>", "hello", "world", "<eos>"],
         "de_tokens": ["<sos>", "hallo", "welt", "<eos>"]
     }
     ```

In [10]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "de_nlp": de_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:

train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

In [12]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)


In [13]:
en_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', 'a', '.', 'in', 'the', 'on', 'man']

In [14]:
de_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', '.', 'ein', 'einem', 'in', 'eine', ',']

In [15]:
en_vocab["the"]

7

In [16]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [17]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)


In [18]:
tokens = ["i", "love", "watching", "crime", "shows"]
en_vocab.lookup_indices(tokens)

[956, 2169, 173, 0, 821]

### 为什么 `crime` 被转换成 `<unk>`？

#### 1. 词汇表的局限性
- 词汇表（`vocab`）是一个预定义的 Token 集合，仅包含模型训练时见过的词。
- 如果某个词（如 `crime`）未出现在训练数据中，则不会被加入词汇表。
- 当 `lookup_indices` 遇到未登录词时，默认返回 `<unk>` 的索引。

#### 2. 可能的原因
- **数据不匹配**：
  - 训练词汇表时未包含 `crime`，但测试/推理时出现了该词。
- **分词策略**：
  - 如果使用基于空格的分词（如 `WhitespaceTokenizer`），`crime` 是一个完整词，但如果词汇表未收录，则会被视为未知词。
- **词汇表大小限制**：
  - 为了控制模型大小，可能人为限制了词汇表的大小（如只保留前 50,000 个高频词），导致低频词被过滤。

#### 3. 解决方案
- **扩展词汇表**：
  - 在训练前收集更多数据，确保词汇表覆盖所有可能的词。
- **使用子词分词**：
  - 改用 BPE、WordPiece 等算法，将未知词拆分为已知子词（如 `crime` → `cr` + `ime`）。
- **处理未知词**：
  - 在模型中为 `<unk>` 分配特殊处理逻辑（如随机初始化向量或使用 `<unk>` 的嵌入）。

### `numericalize_example` 函数的作用

该函数用于将文本数据（Token 列表）转换为数值索引（ID 列表），是 NLP 数据预处理的关键步骤。具体功能如下：

1. **输入参数**：
   - `example`：包含分词后 Token 列表的字典，格式为：
     ```python
     {
         "en_tokens": ["<sos>", "hello", "world", "<eos>"],
         "de_tokens": ["<sos>", "hallo", "welt", "<eos>"]
     }
     ```
   - `en_vocab` 和 `de_vocab`：分别是英语和德语的词汇表对象，提供 `lookup_indices()` 方法将 Token 转换为索引。

2. **处理流程**：
   - 对 `example["en_tokens"]` 中的每个 Token，通过 `en_vocab.lookup_indices()` 查找对应的数值索引，生成 `en_ids` 列表。
   - 对 `example["de_tokens"]` 中的每个 Token，通过 `de_vocab.lookup_indices()` 查找对应的数值索引，生成 `de_ids` 列表。
   - 返回一个新字典，包含数值化后的输入和目标序列：
     ```python
     {
         "en_ids": [1, 2, 3, 4],  # 英语序列的索引表示
         "de_ids": [1, 5, 6, 4]   # 德语序列的索引表示
     }
     ```

3. **输出结果**：
   - 将原始文本数据转换为模型可直接处理的数值形式（ID 序列），便于后续的嵌入层（Embedding Layer）和神经网络计算。

---

### `map` 函数的作用

`map` 是 Hugging Face `datasets` 库提供的批量数据处理方法，用于对数据集中的每个样本应用指定的转换函数。具体作用如下：

1. **功能**：
   - 对 `train_data`、`valid_data` 和 `test_data` 中的每个样本调用 `numericalize_example` 函数。
   - 将分词后的 Token 列表（如 `["<sos>", "hello", "<eos>"]`）转换为数值索引列表（如 `[1, 2, 4]`）。

2. **参数说明**：
   - `numericalize_example`：要应用的转换函数。
   - `fn_kwargs={"en_vocab": en_vocab, "de_vocab": de_vocab}`：传递给函数的额外参数（词汇表对象）。

3. **返回值**：
   - 返回一个新的数据集，其中每个样本已被转换为数值索引形式：
     ```python
     {
         "en_ids": [1, 2, 3, 4],
         "de_ids": [1, 5, 6, 4]
     }
     ```

4. **优势**：
   - **高效性**：`map` 支持多线程并行处理，显著加速大规模数据集的预处理。
   - **灵活性**：通过 `fn_kwargs` 动态传递参数（如不同语言的词汇表），适应多任务需求。

---

### **完整流程示例**
假设原始数据为：
```python
{"en_tokens": ["<sos>", "hello", "world", "<eos>"], 
 "de_tokens": ["<sos>", "hallo", "welt", "<eos>"]}

In [19]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

['i', 'love', 'watching', '<unk>', 'shows']

In [20]:
def numericalize_example(example, en_vocab, de_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

In [21]:
fn_kwargs = {"en_vocab": en_vocab, "de_vocab": de_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:

train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>'],
 'en_ids': [2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3],
 'de_ids': [2, 18, 26, 253, 30, 84, 20, 88, 7, 15, 110, 7647, 3171, 4, 3]}

In [23]:
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [24]:

train_data[0]

{'en_ids': tensor([   2,   16,   24,   15,   25,  778,   17,   57,   80,  202, 1312,    5,
            3]),
 'de_ids': tensor([   2,   18,   26,  253,   30,   84,   20,   88,    7,   15,  110, 7647,
         3171,    4,    3]),
 'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

### `get_collate_fn` 和 `get_data_loader` 函数的作用

#### 1. `get_collate_fn(pad_index)` 函数
- **功能**：创建一个自定义的批处理函数（collate_fn），用于处理变长序列数据（如句子）。
- **输入参数**：
  - `pad_index`：填充符（padding token）的索引值（如 `<pad>` 的 ID）。
- **处理流程**：
  1. 接收一个批次的数据（batch），提取每个样本的英语和德语序列（`en_ids` 和 `de_ids`）。
  2. 使用 `nn.utils.rnn.pad_sequence` 对序列进行填充，使同一批次内的序列长度一致。
  3. 返回填充后的批次数据（包含 `en_ids` 和 `de_ids` 的张量）。
- **作用**：
  - 解决变长序列无法直接批量输入神经网络的问题。
  - 确保同一批次内的序列具有相同的长度（通过填充实现）。

#### 2. `get_data_loader(dataset, batch_size, pad_index, shuffle=False)` 函数
- **功能**：创建一个 PyTorch 的 `DataLoader`，用于分批次加载数据并应用自定义的批处理函数。
- **输入参数**：
  - `dataset`：已预处理的数据集（如 `train_data`）。
  - `batch_size`：每个批次的样本数量。
  - `pad_index`：填充符的索引值（传递给 `get_collate_fn`）。
  - `shuffle`：是否在每个 epoch 开始时打乱数据顺序（默认为 `False`）。
- **处理流程**：
  1. 调用 `get_collate_fn(pad_index)` 获取自定义的批处理函数。
  2. 创建 `DataLoader`，指定数据集、批次大小、批处理函数和是否打乱数据。
- **作用**：
  - 高效加载数据，支持多线程并行处理。
  - 自动将数据分批次，并通过 `collate_fn` 处理变长序列。

#### 关键点
- **填充（Padding）**：确保同一批次内的序列长度一致，是 NLP 中处理变长序列的常见方法。
- **DataLoader**：PyTorch 中用于高效加载数据的工具，支持多线程和自动批处理。
- **适用场景**：适用于序列到序列（Seq2Seq）任务，如机器翻译、文本生成等。

In [25]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

In [26]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader
get_collate_fn(pad_index)

<function __main__.get_collate_fn.<locals>.collate_fn(batch)>

In [27]:

batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)


### `Encoder` 类的作用与实现解析

#### 1. 输入参数
- `input_dim`：输入词汇表的大小（即有多少个不同的词）。
- `embedding_dim`：词嵌入向量的维度（即每个词被表示为多少维的向量）。
- `hidden_dim`：LSTM 隐藏层的维度（即隐藏状态向量的大小）。
- `n_layers`：LSTM 的层数（堆叠多少层 LSTM）。
- `dropout`：Dropout 概率，用于防止过拟合。

#### 2. 核心组件
- **词嵌入层 (`self.embedding`)**：
  - 将输入的整数序列（词索引）转换为密集向量表示（词嵌入）。
  - 输入形状：`[src length, batch size]`（句子长度 × 批次大小）。
  - 输出形状：`[src length, batch size, embedding_dim]`。

- **LSTM 层 (`self.rnn`)**：
  - 处理词嵌入序列，生成隐藏状态和细胞状态。
  - 参数：
    - `embedding_dim`：输入维度（词嵌入的维度）。
    - `hidden_dim`：隐藏层维度。
    - `n_layers`：LSTM 的层数。
    - `dropout`：Dropout 概率（仅用于非输入层之间的连接）。
  - 输出：
    - `outputs`：所有时间步的隐藏状态（通常只用最后一个时间步的隐藏状态）。
    - `hidden` 和 `cell`：最终的隐藏状态和细胞状态（用于传递给 Decoder）。

- **Dropout 层 (`self.dropout`)**：
  - 在词嵌入后应用 Dropout，随机丢弃部分神经元以防止过拟合。

#### 3. `forward` 函数的处理流程
1. **输入处理**：
   - 输入 `src` 的形状为 `[src length, batch size]`（句子长度 × 批次大小）。
   - 例如：`src = [[1, 2, 3], [4, 5, 0]]`（两个句子，第一个句子长度为 3，第二个句子长度为 2，`0` 是填充符）。

2. **词嵌入**：
   - 通过 `self.embedding(src)` 将词索引转换为词嵌入向量。
   - 输出形状：`[src length, batch size, embedding_dim]`。

3. **Dropout**：
   - 对词嵌入结果应用 Dropout，随机丢弃部分神经元。

4. **LSTM 处理**：
   - 将 dropout 后的词嵌入输入到 LSTM 中。
   - LSTM 返回：
     - `outputs`：所有时间步的隐藏状态（通常不需要，除非需要所有时间步的输出）。
     - `hidden` 和 `cell`：最终的隐藏状态和细胞状态（用于传递给 Decoder）。

5. **输出**：
   - 返回 `hidden` 和 `cell`，形状分别为：
     - `hidden`：`[n layers * n directions, batch size, hidden dim]`。
     - `cell`：`[n layers * n directions, batch size, hidden dim]`。
   - 在单层、单向 LSTM 中，`n layers=1`，`n directions=1`，因此形状为 `[1, batch size, hidden dim]`。

#### 4. 输出说明
- **`hidden`**：
  - 最终的隐藏状态，包含整个输入序列的上下文信息。
  - 形状：`[n layers * n directions, batch size, hidden dim]`。
  - 在单层、单向 LSTM 中，`hidden` 的第一个维度是 `1`（因为只有一层），可以简化为 `[batch size, hidden dim]`（如果只需要最后一层的状态）。

- **`cell`**：
  - 最终的细胞状态，与 `hidden` 类似，但用于 LSTM 的内部计算。
  - 形状与 `hidden` 相同。

#### 5. 典型用途
- 在 Seq2Seq 模型中，`Encoder` 的 `hidden` 和 `cell` 状态会被传递给 `Decoder`，作为解码的初始状态。
- 由于 `Encoder` 是双向 LSTM（如果 `n_directions=2`），则 `hidden` 和 `cell` 会拼接来自前向和后向的隐藏状态，以捕获完整的上下文信息。

#### 6. 注意事项
- 如果 `n_layers > 1` 或 `n_directions > 1`，`hidden` 和 `cell` 的第一个维度会相应增加（`n layers * n directions`）。
- 在实际应用中，可能需要调整 `hidden` 和 `cell` 的形状以适应 Decoder 的输入要求（例如，提取最后一层的状态）。

In [28]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

### Decoder 类的工作流程

#### 1. 输入参数说明
- `output_dim`：输出词汇表的大小（即解码器可能生成的词的数量）
- `embedding_dim`：词嵌入向量的维度
- `hidden_dim`：LSTM 隐藏层的维度
- `n_layers`：LSTM 的层数
- `dropout`：Dropout 概率

#### 2. 核心组件解析
- **词嵌入层 (`self.embedding`)**：
  - 将输入的整数（词索引）转换为密集向量表示
  - 输入形状：[batch size]
  - 输出形状：[1, batch size, embedding_dim]（经过 unsqueeze 处理）

- **LSTM 层 (`self.rnn`)**：
  - 处理嵌入后的词向量序列
  - 接收 Encoder 传递的 hidden 和 cell 状态作为初始状态
  - 输出新的 hidden 和 cell 状态

- **全连接层 (`self.fc_out`)**：
  - 将 LSTM 的输出映射到输出词汇表大小的向量空间
  - 用于预测下一个词的概率分布

- **Dropout 层 (`self.dropout`)**：
  - 在词嵌入后应用 Dropout 防止过拟合

#### 3. 前向传播流程
1. **输入处理**：
   - 输入 `input` 的形状为 [batch size]（单个词索引）
   - 通过 unsqueeze(0) 增加时间步维度，变为 [1, batch size]

2. **词嵌入**：
   - 将输入词索引转换为词嵌入向量
   - 应用 Dropout 后形状为 [1, batch size, embedding_dim]

3. **LSTM 处理**：
   - 接收词嵌入向量和 Encoder 的 hidden/cell 状态
   - 输出新的 hidden 和 cell 状态
   - 在单层单向 LSTM 中，hidden/cell 形状为 [n layers, batch size, hidden_dim]

4. **预测输出**：
   - 将 LSTM 输出通过全连接层映射到输出词汇表空间
   - 去除时间步维度后形状为 [batch size, output_dim]
   - 表示每个词的概率分布

#### 4. 关键点说明
- **输入与输出**：
  - 输入：当前时间步的词索引 + Encoder 的上下文状态
  - 输出：下一个词的概率分布 + 更新后的 hidden/cell 状态

- **状态传递**：
  - Decoder 维护自己的隐藏状态和细胞状态
  - 每个时间步都会更新这些状态并传递给下一个时间步

- **与 Encoder 的交互**：
  - 在第一个时间步，Decoder 使用 Encoder 的 final hidden/cell 状态
  - 后续时间步使用前一个时间步的 hidden/cell 状态

- **应用场景**：
  - 在 Seq2Seq 模型中用于生成目标序列
  - 每个时间步生成一个词，直到遇到结束标记或达到最大长度

#### 5. 注意事项
- 该实现假设是单层单向 LSTM（n_layers=1, n_directions=1）
- 如果需要处理多方向 LSTM，需要调整 hidden/cell 的维度处理
- 在训练时通常使用 teacher forcing 技术提高稳定性v

In [29]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

### Seq2Seq 类的工作原理与 Teacher Forcing 机制

#### 1. 类结构概述
- **核心组件**：
  - `encoder`：编码器模块，负责将源语言序列编码为上下文表示
  - `decoder`：解码器模块，基于编码器输出和目标序列生成预测
  - `device`：设备配置（CPU/GPU）

- **关键约束**：
  - 编码器和解码器的隐藏层维度必须相同 
  - 编码器和解码器的层数必须相同

#### 2. forward 函数流程解析
1. **输入参数**：
   - `src`：源语言序列 [src length, batch size]
   - `trg`：目标语言序列 [trg length, batch size]
   - `teacher_forcing_ratio`：使用真实标签作为输入的概率（0-1）

2. **初始化阶段**：
   - 获取批次大小和目标序列长度
   - 创建输出张量 `outputs` 用于存储所有时间步的预测结果
   - 通过编码器处理源序列，获取初始隐藏状态和细胞状态

3. **解码循环**：
   - 从 `<sos>` 标记开始解码
   - 对每个时间步 t：
     a. 将当前输入、隐藏状态和细胞状态传入解码器
     b. 获取预测结果和更新后的隐藏状态/细胞状态
     c. 将预测结果存入 `outputs` 张量
     d. 根据 teacher forcing 决定下一步输入：
        - 以 `teacher_forcing_ratio` 概率使用真实标签 `trg[t]`
        - 否则使用模型预测的最高概率词 `top1`

4. **输出结果**：
   - 返回形状为 [trg length, batch size, output dim] 的张量
   - 包含目标序列每个时间步的预测分布

#### 3. Teacher Forcing 机制详解
- **定义**：
  - 在训练阶段，解码器在生成第 t 个词时，有概率使用真实的目标词（而非模型预测的词）作为输入
  - 概率由 `teacher_forcing_ratio` 控制

- **工作流程**：
  ```python
  teacher_force = random.random() < teacher_forcing_ratio
  input = trg[t] if teacher_force else top1

In [30]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [31]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 编码器初始化
encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)
# 解码器初始化
decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)
# Seq2Seq模型整合
model = Seq2Seq(encoder, decoder, device).to(device)

In [32]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [33]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 13,898,501 trainable parameters


In [34]:
optimizer = optim.Adam(model.parameters())

In [35]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [36]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    # 设置模型为训练模式
    model.train()
    # 初始化当前epoch的总损失
    epoch_loss = 0
    # 遍历数据加载器中的每个批次
    for i, batch in enumerate(data_loader):
        # 获取源语言序列并移动到指定设备
        src = batch["de_ids"].to(device)
        # 获取目标语言序列并移动到指定设备
        trg = batch["en_ids"].to(device)
        # src的形状为 [src length, batch size]
        # trg的形状为 [trg length, batch size]
        
        # 清空优化器的梯度
        optimizer.zero_grad()
        # 将源序列和目标序列输入模型，获取输出
        output = model(src, trg, teacher_forcing_ratio)
        # output的形状为 [trg length, batch size, trg vocab size]
        
        # 获取目标词汇表的大小
        output_dim = output.shape[-1]
        # 去掉输出的第一个时间步（<sos>），并调整形状以适配损失函数
        output = output[1:].view(-1, output_dim)
        # output的形状为 [(trg length - 1) * batch size, trg vocab size]
        
        # 去掉目标序列的第一个时间步（<sos>），并调整形状以适配损失函数
        trg = trg[1:].view(-1)
        # trg的形状为 [(trg length - 1) * batch size]
        
        # 计算当前批次的损失
        loss = criterion(output, trg)
        # 反向传播计算梯度
        loss.backward()
        # 对梯度进行裁剪，防止梯度爆炸
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        # 更新模型参数
        optimizer.step()
        # 累加当前批次的损失
        epoch_loss += loss.item()
    
    # 返回当前epoch的平均损失
    return epoch_loss / len(data_loader)



In [37]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [38]:
n_epochs = 1 # 因模型训练对计算资源要求较高，此处只设立了一轮训练。
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [04:59<00:00, 299.49s/it]

	Train Loss:   5.026 | Train PPL: 152.277
	Valid Loss:   4.861 | Valid PPL: 129.150


In [39]:
model.load_state_dict(torch.load("tut1-model.pt"))

<All keys matched successfully>

In [40]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in de_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = de_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [41]:
sentence = test_data[0]["de"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

('Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.',
 'A man in an orange hat starring at something.')

In [42]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [43]:
translation

['<sos>',
 'a',
 'man',
 'in',
 'a',
 'a',
 'shirt',
 'is',
 'a',
 'a',
 'a',
 '.',
 '<eos>']